In [ ]:
from custom_model.models import DGCN, SoftLoss, DERLoss, MSE_scale
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

import torch
from torch.optim.lr_scheduler import StepLR

from custom_model.utils import *
from custom_model.training import *
from custom_model.custom_dataset import *

from absl import logging
logging._warn_preinit_stderr = 0
logging.warning('Worrying Stuff')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

In [ ]:
para = {}
para['time_invertal'] = 5
para['horizon'] = 15
para['observation'] = 20
para['nb_node'] = 193
para['dim_feature'] = 128
A = adjacency_matrix(3)
B = adjacency_matrixq(3, 8)

#years = ['2018']
years = ['2018', '2019']
years_test = ['2019']

In [ ]:
trainset = AMSdataset(years, para, 'train')
validationset = AMSdataset(years, para, 'validation')
validation_loader = DataLoader(validationset, batch_size=16, shuffle=False)
BATCH_SIZE = 16
EPOCH_NUMBER = 40
#loss = DERLoss(0.1).to(device)
loss = MSE_scale().to(device)

In [ ]:
model = DGCN(para, A, B, uncertainty=False).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=1, gamma=0.98)
train_model(EPOCH_NUMBER, BATCH_SIZE, trainset, model, optimizer, validation_loader, loss,
                  scheduler, para['horizon'], beta=-0.07)

In [ ]:
torch.save(model.state_dict(), './pretrained/model_mse.pt')

In [ ]:
model = DGCN(para, A, B, return_interpret=False, uncertainty=False).to(device)
#model.load_state_dict(torch.load('./pretrained/predictor_uq.pt'))
model.load_state_dict(torch.load('./pretrained/model_mse.pt'))

In [ ]:
para['time_invertal'] = 5
testset = AMSdataset(['2022'], para, 'test')
#testset = AMSdataset(years, para, 'train')
prediction = test_run_point(testset, model)

In [ ]:
y = model(torch.Tensor(testset.X[[99]]).float().to(device), 0)

In [ ]:
plt.imshow(y[0,...,0])

In [ ]:
alea, epis = test_run_uq(testset, model)

In [ ]:
epis = test_run_rarity(testset, model)

In [ ]:
weights = np.where(testset.X[:,-15:,:,0]>0.45, 1., 1.)
MAE = np.mean(np.abs(prediction[...,0]-testset.X[:,-15:,:,0])*weights)*130
MAPE = np.mean(weights*np.abs(prediction[...,0]-testset.X[:,-15:,:,0])/testset.X[:,-15:,:,0]*100)
RMSE = np.mean(weights*(prediction[...,0]-testset.X[:,-15:,:,0])**2)**0.5*130

In [ ]:
print(MAE, MAPE, RMSE)

In [ ]:
MAE = np.mean(np.abs(prediction[...,1]-testset.X[:,-15:,:,1])*weights)*3000
MAPE = np.mean(weights*np.abs(prediction[...,1]-testset.X[:,-15:,:,1])/testset.X[:,-15:,:,1]*100)
RMSE = np.mean(weights*(prediction[...,1]-testset.X[:,-15:,:,1])**2)**0.5*3000

In [ ]:
print(MAE, MAPE, RMSE)

In [ ]:
np.mean(testset.X[np.argsort(epis)[-50],...,0])

In [ ]:
np.mean(epis)**0.5*130

In [ ]:
plt.scatter(np.amin(testset.X[np.argsort(epis)][:,-15:,:,0], (1,2)), np.sort(epis), s=0.2)
plt.show()

In [ ]:
np.savez_compressed('./results/order_2021', perserve=np.where(epis**0.5*130>1.78)[0], remove=np.where(epis**0.5*130<=1.78)[0])

In [ ]:
np.savez_compressed('./results/order_train', order=np.argsort(epis))

In [ ]:
np.sort(epis)[int(len(epis)*0.7)]**0.5*130

In [ ]:
np.sort(epis)[-16099]**0.5*130

In [ ]:
fig, ax = plt.subplots(figsize=(5,3))
ax.hist(epis**0.5*130, bins=np.linspace(0, 12, 60), density=True, alpha=0.8, align='left')
plt.xlim(-0.1,10)
plt.ylim(0,1.15)
plt.vlines(2.59, 0, 1.15, colors='red', ls='-.', label='70%, 2.59')
plt.vlines(1.52, 0, 1.15, colors='black', ls='--', label='free-flowing, 1.52')
plt.xlabel('Knowledge uncertainty (km/h)')
plt.ylabel('probability')
plt.legend()
plt.tight_layout()
plt.savefig('./imgs/dist_knowledge_train.pdf', dpi=600)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(5,3))
ax.plot(np.arange(4,44,4), np.mean(alea[:,:10], (0,2))**0.5*130, marker='x', label='Aleatoric')
ax.plot(np.arange(4,44,4), np.mean(epis[:,:10], (0,2))**0.5*130*2, marker='+', label='Epistemic')
ax.plot(np.arange(4,44,4), np.mean(epis[:,:10]+alea[:,:10]*4, (0,2))**0.5*130, marker='*', label='Total')
ax.set_xlabel('prediction horizon (min)')
ax.set_ylabel(r'$\sigma$ (km/h)')
ax.set_xticks(np.arange(4,44,4))
ax.set_xlim(0,44)
ax.set_ylim(0,20)
ax.set_title('Test set 2019')
plt.legend()
plt.grid()
fig.tight_layout()
fig.savefig('./imgs/overfit19.pdf', dpi=600)
plt.show()

In [ ]:
np.mean((prediction[...,1]-testset.X[:,-15:,:,1])**2)**0.5*3000

In [ ]:
weights = np.where(testset.X[:,-15:,:,1]>0.45, 1., 1.)
np.mean(weights*np.abs(prediction[...,1]-testset.X[:,-15:,:,1]))*3000

In [ ]:
np.mean(weights*np.abs(prediction[...,1]-testset.X[:,-15:,:,1])/testset.X[:,-15:,:,1]*100)

In [ ]:
ji = -8
print(inds[ji]//52)
plt.imshow(testset.X[inds[ji],...,0], vmin=0, vmax=1, aspect='auto')
plt.show()

In [ ]:
plt.imshow(np.concatenate((testset.X[inds[ji],:20,...,0], prediction[inds[ji],...,0]), 0), vmin=0, vmax=1, aspect='auto')
plt.show()

In [ ]:
plt.imshow(alea[inds[ji]], aspect='auto')
plt.show()

In [ ]:
#plt.scatter(np.arange(2, 32, 2), np.mean(alea, (0,2))**0.5*130-1)
#plt.scatter(np.arange(2, 32, 2), np.mean(epis, (0,2))**0.5*130-1)
plt.scatter(np.arange(2, 32, 2), np.mean(alea/epis, (0,2)))
#plt.scatter(np.arange(2, 32, 2), s)
plt.show()

In [ ]:
plt.scatter(prediction[:,1,:,0].flatten(), epis[:,1,:].flatten()**0.5)
plt.show()

In [ ]:
dt = np.load('./results/SP.npz', allow_pickle=True)

In [ ]:
plt.scatter(np.mean(prediction[...,0], (1,2)), np.mean(epis, (1,2))**0.5)
plt.show()

In [ ]:
np.mean(alea)**0.5*130, np.mean(epis)**0.5*130, np.mean(epis+alea)**0.5*130